<a href="https://colab.research.google.com/github/itsdevansh/CSI5386-NLP/blob/main/Assignment2/Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
train_df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/NLP-CSI5386/subtaskA_train_monolingual.jsonl', lines=True)
test_df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/NLP-CSI5386/subtaskA_test_monolingual.jsonl', lines=True)
gs_df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/NLP-CSI5386/subtaskA_gs_monolingual.jsonl', lines=True)

In [4]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()) :
  !pip install unsloth
else :
  !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
  !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
  !pip install --no-deps unsloth

In [5]:
from transformers import TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer
from sklearn.metrics import accuracy_score

# Data Prep

In [6]:
def prepare_dataset(texts, labels) :
  formatted_data = []

  for text, label in zip(texts, labels) :

    #Format instruction

    item = {
        "text" : f"<s>[INST] Classify the following text as 0 or 1 : {text} [/INST] {label} </s>"
    }

    formatted_data.append(item)

  return Dataset.from_pandas(pd.DataFrame(formatted_data))


#Split into training and validation data using sklearn train test split
X_train,X_val,y_train,y_val = train_test_split(train_df['text'], train_df['label'], test_size=0.2, random_state=42)
train_dataset = prepare_dataset(X_train.tolist(), y_train.values)
val_dataset = prepare_dataset(X_val.tolist(), y_val.values)

In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model_names = [
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit"]

model_name = model_names[0]
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

<ipython-input-7-9ac25353ef48>:1: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/331k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/7.00G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/6.90G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/6.94G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/6.94G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/6.99G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.75G [00:00<?, ?B/s]

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
model = FastLanguageModel.get_peft_model (
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


In [ ]:
output_dir = "content/drive/MyDrive/Colab Notebooks/NLP-CSI5386"
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,  # Accumulate gradients to simulate larger batch size
    learning_rate=2e-4,             # Higher LR works well with LoRA
    fp16=True,                      # Mixed precision training
    logging_steps=10,
    eval_steps=100,
    save_steps=100,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    report_to="none",
    weight_decay=0.001,             # L2 regularization
    optim="adamw_torch",
    lr_scheduler_type="cosine",     # Cosine LR schedule
    warmup_ratio=0.1,               # Warmup for first 10% of steps
)

In [ ]:
trainer = SFTTrainer (
    model = model,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    tokenizer = tokenizer,
    args = training_args,
    packing = False,
    max_seq_length = max_seq_length,
    dataset_text_field = "text",
)

In [ ]:
trainer.train()

In [ ]:
# Save the model
trainer.save_model(f"{output_dir}/{model_name}")

In [ ]:
def predict(texts, model, tokenizer, device = "cuda") :
  model.eval()
  predictions =[]

  for text in texts :
    prompt = f"<s>[INST] Classify the following text as 0 or 1 : {text} [/INST]"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.to(device)

    with torch.no_grad() :
      outputs = model.generate(**inputs,
                               max_new_tokens=10,
                               temperature = 0.1,
                               do_sample = False
                               )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions.append(decoded)

  return predictions


In [ ]:
test_texts = test_df['text'].tolist()
test_preds = predict(test_texts, model, tokenizer)

In [ ]:
# Save predictions
predictions_file = pd.DataFrame({'id': test_df['id'], 'label': test_preds})
predictions_file.to_json(f'{output_dir}/{model_name}_unsloth_predictions.jsonl', lines=True, orient="records")